In [1]:
import sqlite3 


IMPORTANT: You need a Python kernal that's been compiled that has loadable sqlite extensions enabled. If you don't want to compile it yourself, use the package on Homebrew, which comes with the flag set.

https://til.simonwillison.net/sqlite/sqlite-extensions-python-macos 

In [2]:
con = sqlite3.connect("/Users/Bumpant/Development/LLM/Rag/imdb.db")
con.enable_load_extension(True)
con.load_extension("../sqlite-lembed/lembed0")
con.load_extension("../sqlite-vec/vec0")
curs = con.cursor()


In [3]:
curs.execute("INSERT INTO temp.lembed_models(name, model) select 'nomic-embed-text-1.5', lembed_model_from_file('/Users/Bumpant/Development/LLM/Rag/nomic-embed-text-v1.5.Q2_K.gguf');")


In [4]:
curs.execute("SELECT COUNT(*) FROM reviews")
curs.fetchall()

[(75935,)]

In [5]:
# -- Build a vector table with embeddings of article headlines, using OpenAI's API
curs.execute('DROP TABLE IF EXISTS vec_reviews;')
curs.execute('create virtual table vec_reviews using vec0(review_embeddings float[768]);')



I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, even then it's not shot like some cheaply made porno. While my countrymen mind find it shocking, in reality sex and nudity are a major staple in Swedish cinema. Even Ingmar Bergman, arguably their answer to good old boy John Ford, had sex scenes in his films.<br /><br />I do commend the filmmakers for the fact that any sex shown in the film is shown for artistic purposes rather than just to shock people and make money to be shown in pornographic theaters in America. I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot.|

Who are these "They"- the actors? the filmmakers? Certainly couldn't be the audience- this is among the most air-puffed productions in existence. It's the kind of movie that looks like it was a lot of fun to shoot TOO much fun, nobody is getting any actual work done, and that almost always makes for a movie that's no fun to watch.<br /><br />Ritter dons glasses so as to hammer home his character's status as a sort of doppleganger of the bespectacled Bogdanovich; the scenes with the breezy Ms. Stratten are sweet, but have an embarrassing, look-guys-I'm-dating-the-prom-queen feel to them. Ben Gazzara sports his usual cat's-got-canary grin in a futile attempt to elevate the meager plot, which requires him to pursue Audrey Hepburn with all the interest of a narcoleptic at an insomnia clinic. In the meantime, the budding couple's respective children (nepotism alert: Bogdanovich's daughters) spew cute and pick up some fairly disturbing pointers on 'love' while observing their parents. (Ms. Hepburn, drawing on her dignity, manages to rise above the proceedings- but she has the monumental challenge of playing herself, ostensibly.) Everybody looks great, but so what? It's a movie and we can expect that much, if that's what you're looking for you'd be better off picking up a copy of Vogue.<br /><br />Oh- and it has to be mentioned that Colleen Camp thoroughly annoys, even apart from her singing, which, while competent, is wholly unconvincing... the country and western numbers are woefully mismatched with the standards on the soundtrack. Surely this is NOT what Gershwin (who wrote the song from which the movie's title is derived) had in mind; his stage musicals of the 20's may have been slight, but at least they were long on charm. "They All Laughed" tries to coast on its good intentions, but nobody- least of all Peter Bogdanovich - has the good sense to put on the brakes.<br /><br />Due in no small part to the tragic death of Dorothy Stratten, this movie has a special place in the heart of Mr. Bogdanovich- he even bought it back from its producers, then distributed it on his own and went bankrupt when it didn't prove popular. His rise and fall is among the more sympathetic and tragic of Hollywood stories, so there's no joy in criticizing the film... there _is_ real emotional investment in Ms. Stratten's scenes. But "Laughed" is a faint echo of "The Last Picture Show", "Paper Moon" or "What's Up, Doc"- following "Daisy Miller" and "At Long Last Love", it was a thundering confirmation of the phase from which P.B. has never emerged.<br /><br />All in all, though, the movie is harmless, only a waste of rental. I want to watch people having a good time, I'll go to the park on a sunny day. For filmic expressions of joy and love, I'll stick to Ernest Lubitsch and Jaques Demy...|

In [6]:
processedRecords = 0
totalRecords = 75935
recordsPerBatch = 50

while totalRecords > processedRecords:
    curs.execute("\
                INSERT INTO vec_reviews(rowid, review_embeddings) \
                    SELECT rowid, lembed('nomic-embed-text-1.5', review) \
                    FROM reviews WHERE review is not null \
                    LIMIT "  + str(recordsPerBatch) + " OFFSET "+ str(processedRecords) + ";")
    
    processedRecords += recordsPerBatch
    print("Processed records: " + str(processedRecords) + " out of " + str(totalRecords) + " records")
    


Processed records: 50 out of 75935 records
Processed records: 100 out of 75935 records


In [ ]:
curs.execute('select * from reviews;')
print(curs.fetchone())

('<br /><br />Oh- and it has to be mentioned that Colleen Camp thoroughly annoys, even apart from her singing, which, while competent, is wholly unconvincing... the country and western numbers are woefully mismatched with the standards on the soundtrack. Surely this is NOT what Gershwin (who wrote the song from which the movie\'s title is derived) had in mind; his stage musicals of the 20\'s may have been slight, but at least they were long on charm. "They All Laughed" tries to coast on its good intentions, but nobody- least of all Peter Bogdanovich - has the good sense to put on the brakes.<br /><br />Due in no small part to the tragic death of Dorothy Stratten, this movie has a special place in the heart of Mr. Bogdanovich- he even bought it back from its producers, then distributed it on his own and went bankrupt when it didn\'t prove popular. His rise and fall is among the more sympathetic and tragic of Hollywood stories, so there\'s no joy in criticizing the film... there _is_ rea

In [ ]:
# curs.execute("with matches as (select rowid, distance from vec_reviews where review_embeddings match lembed('nomic-embed-text-1.5', 'terrible film') order by distance limit 3) select review, distance from matches left join reviews on reviews.rowid = matches.rowid;")

# print(curs.fetchall())

con.close()
